In [6]:
import numpy as np
import librosa
import pandas as pd
import freesound

print("Librosa version:", librosa.__version__)
print("Numpy version:", np.__version__)
print("Pandas version:", pd.__version__)
#print(freesound.__version__)

Librosa version: 0.11.0
Numpy version: 1.26.4
Pandas version: 2.3.0


In [3]:
import soundata

# Inicializace datasetu
dataset = soundata.initialize("urbansound8k")

# Stáhne dataset, pokud ještě není
print("📦 Kontroluji dataset...")
dataset.download()  

# Načte metadata
dataset.validate()
clip_ids = dataset.clip_ids

print(f"🔍 V databázi je {len(clip_ids)} klipů.")

INFO: Downloading ['all', 'index']. Index is being stored in /home/cigi/soundenv/lib/python3.10/site-packages/soundata/datasets/indexes, and the rest of files in /home/cigi/sound_datasets/urbansound8k
INFO: [all] downloading UrbanSound8K.tar.gz
INFO: /home/cigi/sound_datasets/urbansound8k/UrbanSound8K.tar.gz already exists and will not be downloaded. Rerun with force_overwrite=True to delete this file and force the download.


📦 Kontroluji dataset...


INFO: /home/cigi/sound_datasets/urbansound8k/UrbanSound8K_README.txt already exists. Run with force_overwrite=True to download from scratch
INFO: /home/cigi/sound_datasets/urbansound8k/audio already exists. Run with force_overwrite=True to download from scratch
INFO: /home/cigi/sound_datasets/urbansound8k/FREESOUNDCREDITS.txt already exists. Run with force_overwrite=True to download from scratch
INFO: /home/cigi/sound_datasets/urbansound8k/metadata already exists. Run with force_overwrite=True to download from scratch
INFO: [index] downloading urbansound8k_index_1.0.json
INFO: /home/cigi/soundenv/lib/python3.10/site-packages/soundata/datasets/indexes/urbansound8k_index_1.0.json already exists and will not be downloaded. Rerun with force_overwrite=True to delete this file and force the download.
100%|██████████| 8732/8732 [00:17<00:00, 502.85it/s]
INFO: Success: the dataset is complete and all files are valid.
INFO: --------------------


🔍 V databázi je 8732 klipů.


In [4]:
def extract_mfcc(path, n_mfcc=13, duration=4.0, sr=22050):
    y, _ = librosa.load(path, sr=sr, duration=duration)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    return np.mean(mfcc.T, axis=0)  # průměr přes časové okno

# Kategorie
target_labels = ['gun_shot']
negative_labels = ['children_playing', 'air_conditioner', 'dog_bark']

X = []
y = []

for clip_id in dataset.clip_ids:
    clip = dataset.clip(clip_id)
    if clip.tags:
        label = clip.tags.labels[0]

        if label in target_labels + negative_labels:
            features = extract_mfcc(clip.audio_path)
            X.append(features)
            y.append(1 if label == 'gun_shot' else 0)

X = np.array(X)
y = np.array(y)

print("✅ Dataset hotový:", X.shape, y.shape)

✅ Dataset hotový: (3374, 13) (3374,)


In [2]:
# 🔑 Vlož sem svůj Freesound API klíč
API_KEY = "VV7pMu2KzRHPQkBZAFeSg96sH4hPdHi6OMryNnCc"

In [ ]:
import freesound
import os

API_KEY = "VV7pMu2KzRHPQkBZAFeSg96sH4hPdHi6OMryNnCc"

client = freesound.FreesoundClient()
client.set_token(API_KEY, "token")

# Výstupní složka
os.makedirs("freesound_gunshots", exist_ok=True)

# 🔎 Vyhledání gunshot zvuků (s výpisem)
print("🔍 Hledám zvuky 'gunshot' s licencí CC0...")
results = list(client.text_search(
    query="gunshot",
    filter="license:cc-by OR license:cc0",
    fields="id,name,previews,license",
    page_size=20,
    sort="score"
))

# ✅ Výpis výsledků
print(f"\n🧾 Nalezeno {len(results)} výsledků:\n")

for i, sound in enumerate(results):
    print(f"[{i+1}] {sound.name} | ID: {sound.id} | Licence: {sound.license}")

if len(results) == 0:
    print("⚠️ Žádné výsledky nenalezeny! Zkus jiný dotaz nebo zkontroluj API klíč.")
    exit()

# ⬇️ Stažení zvuků
print("\n📥 Spouštím stahování zvuků...")
for i, sound in enumerate(results):
    safe_name = f"{sound.id}_{sound.name.replace(' ', '_')}.mp3"
    path = os.path.join("freesound_gunshots", safe_name)
    print(f"[{i+1}] ⬇️ Stahuji: {safe_name}")
    try:
        sound.retrieve_preview(path)
    except Exception as e:
        print(f"❌ Chyba při stahování {safe_name}: {e}")

print("\n✅ Hotovo! Zkontroluj složku 'freesound_gunshots'.")

🔍 Hledám zvuky 'gunshot' s licencí CC0...

🧾 Nalezeno 0 výsledků:

⚠️ Žádné výsledky nenalezeny! Zkus jiný dotaz nebo zkontroluj API klíč.

📥 Spouštím stahování zvuků...

✅ Hotovo! Zkontroluj složku 'freesound_gunshots'.


: 

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

# Rozdělení dat
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Trénink modelu
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Vyhodnocení
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.97      0.99      0.98       612
           1       0.92      0.71      0.80        63

    accuracy                           0.97       675
   macro avg       0.94      0.85      0.89       675
weighted avg       0.97      0.97      0.97       675

